In [86]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sklearn as sklearn
import statsmodels.formula.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
import sklearn.metrics as metrics
import itertools
from sklearn.metrics import mean_squared_error

In [87]:
df = pd.read_csv("cdc_clean_carefully_dropped_missing.csv")
df.head()

,cnty_fips,county,state,cardiac_intensive_care_units,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,...,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
0,2020,Anchorage,AK,1.0,13.5,23.6,24.3,1.0,72.5,18.7,...,5.2,9.1,293531.0,57.9,0.430,0.430,83.0,9.5,4.4,3.0
1,2090,Fairbanks North Star,AK,1.0,13.0,29.6,33.6,1.0,70.8,17.1,...,4.2,8.0,99072.0,69.9,0.409,0.409,36.0,7.2,4.7,3.0
2,2122,Kenai Peninsula,AK,0.0,15.3,25.7,33.6,1.0,75.4,18.8,...,0.5,4.1,58464.0,79.9,0.439,0.439,52.0,10.4,6.0,4.0
3,2170,Matanuska-Susitna,AK,0.0,15.3,24.1,22.7,0.0,72.3,17.6,...,1.1,4.9,105369.0,79.2,0.422,0.422,26.0,9.6,6.1,3.0
4,6001,Alameda,CA,3.0,10.6,26.2,23.6,6.0,80.8,13.8,...,10.3,22.3,1656754.0,31.4,0.462,0.462,84.0,8.9,3.0,1.0


Selecting variables of interest for EDA

In [88]:
df = df[["prevalence", "cholesterol_awareness", "diagnosed_diabetes", "obesity",  "leisure_inactivity", "smoking", \
        "cardiac_rehab_eligibility", "cholesterol_medication", "cholesterol_screen", "cardiac_rehab_sessions", \
         "cardiac_rehab_particpation", "cardiac_rehab_units", "total_healthcare_cost_per_capita", \
         "air_quality", "park_access", "population_age_65_plus", "population_total", "urban_rural_class", \
         "median_houshld_income", "poverty_rate"]]

## Exploratory Regression Anlaysis

Fit a multiple linear regression with prevalence and socio-economic factors

In [89]:
from sklearn.preprocessing import StandardScaler

dfp = df.copy()
scaler = StandardScaler()
scaler.fit(dfp)
dfp_scaled = scaler.transform(dfp)
dfp_scaled = pd.DataFrame(dfp_scaled)
dfp_scaled.columns = dfp.columns
dfp.describe()

,prevalence,cholesterol_awareness,diagnosed_diabetes,obesity,leisure_inactivity,smoking,cardiac_rehab_eligibility,cholesterol_medication,cholesterol_screen,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,total_healthcare_cost_per_capita,air_quality,park_access,population_age_65_plus,population_total,urban_rural_class,median_houshld_income,poverty_rate
count,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2.310000e+03,2310.000000,2310.000000,2310.000000
mean,8.279524,36.738831,10.382554,33.489221,25.020303,19.748874,17.168918,15.530563,79.856537,24.616840,34.218615,0.981818,19468.945455,7.856504,26.330736,18.478615,1.322281e+05,3.344589,0.443936,13.370260
std,1.657266,3.442488,3.197209,5.422442,5.512959,3.644148,4.748979,2.894919,2.998983,4.779962,16.275084,1.474665,2226.215413,1.582564,22.012412,4.389593,3.201644e+05,0.831313,0.032838,4.854305
min,3.800000,23.900000,2.200000,11.000000,8.800000,6.900000,6.300000,9.600000,60.300000,2.000000,1.200000,0.000000,12283.000000,1.800000,0.000000,6.600000,1.564000e+03,1.000000,0.343000,2.700000
25%,7.100000,34.500000,8.100000,30.100000,21.200000,17.300000,13.900000,13.500000,78.200000,21.900000,22.200000,0.000000,17905.250000,6.900000,9.000000,15.700000,1.865600e+04,3.000000,0.421000,9.900000
50%,8.300000,37.100000,9.900000,33.800000,24.700000,19.600000,16.400000,15.300000,80.000000,25.200000,32.300000,1.000000,19306.000000,8.100000,20.000000,18.200000,3.953550e+04,4.000000,0.442000,12.700000
75%,9.400000,39.200000,12.200000,37.200000,28.700000,22.275000,19.800000,17.200000,81.800000,27.800000,43.700000,1.000000,20757.500000,8.900000,38.750000,20.700000,1.033755e+05,4.000000,0.465750,16.100000
max,14.500000,46.000000,28.700000,53.500000,44.400000,32.600000,51.200000,38.200000,90.100000,40.000000,99.800000,26.000000,30632.000000,16.000000,100.000000,56.700000,5.198275e+06,4.000000,0.601000,36.600000


**Combined multiple regression analysis**

In [91]:
formula = 'prevalence ~ cholesterol_awareness + diagnosed_diabetes + obesity + leisure_inactivity + smoking + \
        cardiac_rehab_eligibility + cholesterol_medication + cholesterol_screen + cardiac_rehab_sessions + \
         cardiac_rehab_particpation + cardiac_rehab_units + total_healthcare_cost_per_capita + \
         air_quality + park_access + population_age_65_plus + population_total + urban_rural_class + \
         median_houshld_income + poverty_rate'

model = sm.ols(formula = formula, data = dfp)
fitted = model.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:             prevalence   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     1794.
Date:                Fri, 04 Mar 2022   Prob (F-statistic):               0.00
Time:                        23:49:08   Log-Likelihood:                -1250.3
No. Observations:                2310   AIC:                             2541.
Df Residuals:                    2290   BIC:                             2656.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep



Investigating the summary of the Multiple Linear Regression, we see that the R-squared value is 0.937 suggesting that 93.7% of the variations in the target varaiable, "prevalence", can be explained by the variations in the predictor variables. Furthermore, the fact that the F-statistic is far far greater than the P-value (0.00) suggests that we cannot but conclude that there exist some relationship between prevalence and the predictor variables. 

Nevertheless, by inspecting the individual p-values of the coefficients of the predictors, it is observed that "diagnosed_diabetes", "obesity", "total_healthcare_cost_per_capita" and "park_access" with p-values 0.960, 0.143, 0.191 and 0.456 respectively, all appear to larger p-values than alpha, considering alpha significant level of 0.05 thereby suggesting that their resulting coefficients are insignificant in the model. While it may be argued that the variables from the surface should significantly influence the model, these observations suggest that it is possible that some other variables, which are correlated with these four variables, are accounting for the variations in the model significantly that the effects of these four variables become insignificant.


Based on these findings, we penalized these four varaiables and built a predictive "linear regression" model and evaluated the performance of the model with new dataset (i.e. test sets).


We first split the actual data to train and test sets, and used the train sets to train a linear regression model. We name this LRModel-1. Then, we used test sets to check how well the model generalize to new datasets. We compare the results with those obtained from other models.

In [92]:
#

### Predictive Regression Analysis (Model)

In [93]:
#Needed Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import itertools
from sklearn.metrics import mean_squared_error

In [94]:
def scaler_function(data):
  
    #Sclaing our data for the univariate mode
    scaler = StandardScaler()
    scaler.fit(data)
    data = scaler.transform(data)
  
    return data

In [95]:
def get_data_split(x_data,y_data):

    #splitting the data into train test split
    x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x_data, y_data, test_size=0.25, random_state=42)

    return x_train_data, x_test_data, y_train_data, y_test_data

In [96]:
def linear_model(x_data, y_data):
    # Convert features and target to numpy array.
    y = np.array(y_data)
    x = np.array(x_data)
    
    #Calling the linear regression model based on x and y data
    lin_model = LinearRegression(fit_intercept=True)
    lin_model.fit(x_data,y_data) 

    return lin_model

In [97]:
def print_model_stats(model):
  
    #Printing model results and coefficients.
    print("Beta Coefficient:", model.coef_)
    print("Model R2:", metrics.r2_score(y_test_data, reg_model.predict(x_test_data)))
    print("Model Training MSE:", np.mean((model.predict(x_train_data) - y_train_data)**2))
    print("Model Testing MSE:", np.mean((model.predict(x_test_data) - y_test_data)**2))
    

In [98]:
def findsubsets(Set,m):
    return set(itertools.combinations(Set, m))

**First, we convert "urban-rural-class" to dummy variables**

In [99]:
df["urban_rural_class"] = df["urban_rural_class"].map({1.0:'large_central_metro', 2.0:'large_fringe_metro', 3.0:'small_metro', 4.0:'nonmetro'})
df_with_dummies = pd.get_dummies(df, columns = ["urban_rural_class"], drop_first=True)
print(df_with_dummies.columns)
print(len(df_with_dummies.columns))

Index(['prevalence', 'cholesterol_awareness', 'diagnosed_diabetes', 'obesity',
       'leisure_inactivity', 'smoking', 'cardiac_rehab_eligibility',
       'cholesterol_medication', 'cholesterol_screen',
       'cardiac_rehab_sessions', 'cardiac_rehab_particpation',
       'cardiac_rehab_units', 'total_healthcare_cost_per_capita',
       'air_quality', 'park_access', 'population_age_65_plus',
       'population_total', 'median_houshld_income', 'poverty_rate',
       'urban_rural_class_large_fringe_metro', 'urban_rural_class_nonmetro',
       'urban_rural_class_small_metro'],
      dtype='object')
22


In [100]:
df_with_dummies

,prevalence,cholesterol_awareness,diagnosed_diabetes,obesity,leisure_inactivity,smoking,cardiac_rehab_eligibility,cholesterol_medication,cholesterol_screen,cardiac_rehab_sessions,...,total_healthcare_cost_per_capita,air_quality,park_access,population_age_65_plus,population_total,median_houshld_income,poverty_rate,urban_rural_class_large_fringe_metro,urban_rural_class_nonmetro,urban_rural_class_small_metro
0,5.1,32.1,8.6,30.1,17.0,17.3,13.5,18.7,72.5,23.6,...,16889,6.400,83.0,10.5,293531.0,0.430,9.5,0,0,1
1,4.8,31.6,7.9,31.1,18.6,17.5,13.0,17.1,70.8,29.6,...,12996,13.900,36.0,9.8,99072.0,0.409,7.2,0,0,1
2,7.4,35.1,8.1,32.9,19.9,20.8,15.3,18.8,75.4,25.7,...,18183,8.125,52.0,16.5,58464.0,0.439,10.4,0,1,0
3,5.9,32.3,8.6,34.3,19.8,20.5,15.3,17.6,72.3,24.1,...,17259,5.700,26.0,11.3,105369.0,0.422,9.6,0,0,1
4,4.7,28.4,6.7,22.0,14.5,11.3,10.6,13.8,80.8,26.2,...,20204,8.700,84.0,13.5,1656754.0,0.462,8.9,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,10.3,42.2,10.0,35.0,23.5,21.5,12.8,13.2,83.8,18.2,...,16842,5.600,39.0,25.3,16399.0,0.497,12.7,0,1,0
2306,8.3,39.3,10.8,35.4,28.4,21.4,17.0,11.3,81.9,22.0,...,17718,7.100,26.0,20.7,40904.0,0.419,9.6,0,1,0
2307,8.3,38.9,8.3,37.4,26.7,21.0,13.1,12.3,81.0,29.7,...,18765,5.300,20.0,18.8,20325.0,0.416,9.7,0,1,0
2308,10.1,42.7,8.3,29.3,19.7,17.6,16.5,12.7,85.7,28.7,...,15211,5.400,53.0,30.5,21751.0,0.478,11.2,0,1,0


**Model 1 (MLR-Model)**

For this model, remember, we drop the variables with high p-values including "diagnosed_diabetes", "obesity", "total_healthcare_cost_per_capita" and "park_access". Then we split into training and test sets, scale the sets, and fit and evaluate model with them.

In [101]:
#Defining predictors and target variables
X = df_with_dummies.drop(["prevalence", "diagnosed_diabetes", "obesity", "total_healthcare_cost_per_capita", \
                          "park_access"], axis = 1)

y = df_with_dummies["prevalence"]

X.shape

(2310, 17)

In [102]:
#Split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape

(1732, 17)

In [103]:
# scale the predictors
X_train_scaled = scaler_function(X_train)
X_test_scaled = scaler_function(X_test)
# y_train_scaled = scaler_function(pd.DataFrame(y_train))
# y_test_scaled = scaler_function(pd.DataFrame(y_test))

In [104]:
X_train_scaled

array([[ 0.05870319,  1.84161231, -0.00877494, ..., -0.4145526 ,
        -1.09451795,  1.59889592],
       [-1.45370705, -0.89782008, -1.3045359 , ..., -0.4145526 ,
        -1.09451795,  1.59889592],
       [ 0.34406361,  0.36376063,  0.28817028, ..., -0.4145526 ,
        -1.09451795,  1.59889592],
       ...,
       [-0.08397702, -0.64550394, -0.06276498, ..., -0.4145526 ,
        -1.09451795,  1.59889592],
       [ 0.08723923,  0.94048323, -0.08976   , ..., -0.4145526 ,
        -1.09451795,  1.59889592],
       [-0.08397702,  0.30969288, -0.27872514, ..., -0.4145526 ,
        -1.09451795,  1.59889592]])

In [105]:
# train model
mlr_model = linear_model(X_train_scaled, y_train)

In [108]:
# Evaluate model
r_squared = mlr_model.score(X_test_scaled, y_test)                       # R^2

y_predicted = mlr_model.predict(X_test_scaled)

test_error = mean_squared_error(y_test, y_predicted, squared=False)      # Root Mean Sqaured Error

print('R-squared' + ":", round(r_squared, 4))
print("predcition error:", round(test_error, 6))

R-squared: 0.9173
predcition error: 0.439141


This model predicts prevalence by an error of 0.4391 with 91.73% of variations in prevalence explained by the variations in the risk factors, healthcare opportunities and socio-economic factors.


In this first model, we eliminated some of the variables based on the findings from the initial exploratory model due to the high p-values of those variables in that model. As an alternative, we implmented Lasso Regression. The aim of lasso regression is to determine the subset of predictors that minimizes prediction error of the target variable. We call this model lasso_model.

The Lasso regression requires a lambda parameter called lasso pararmeter which determines how the coefficients of the variables should be penalized. To detremine the best Lasso parameter, we have to tune this paramter by training different Lasso model and select the one with the best $R^2$.

**Model 2 (lasso_model)**

In [109]:
#Defining predictors and target variables
X = df_with_dummies.drop("prevalence", axis = 1)

y = df_with_dummies["prevalence"]

X.shape

(2310, 21)

In [110]:
#Split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape

(1732, 21)

In [111]:
# sacle the predictors
X_train_scaled = scaler_function(X_train)
X_test_scaled = scaler_function(X_test)
# y_train_scaled = scaler_function(pd.DataFrame(y_train))
# y_test_scaled = scaler_function(pd.DataFrame(y_test))

In [112]:
# Train lasso models with different laaso parameters
N_alphas = 10
alphas = np.exp(np.linspace(np.log(0.01),np.log(1),N_alphas))
models = []
r2_scores = []
for i in range(N_alphas):
    model_i = Lasso(alphas[i])      
    model_i.fit(X_train_scaled,y_train)
    models.append(model_i)
    
    r2_score = model_i.score(X_test_scaled, y_test)
    r2_scores.append(r2_score)
    
best_r2_score_index = r2_scores.index(max(r2_scores))
best_alpha = list(alphas)[best_r2_score_index]
print("best alpha:", round(best_alpha,4))

#best Lasso model
lasso_model = models[best_r2_score_index]

best alpha: 0.0464


In [118]:
#best predictors

best_indices = [index for index,beta in enumerate(lasso_model.coef_) if beta > 0 or beta < 0 ]
X.columns[best_indices]

Index(['cholesterol_awareness', 'leisure_inactivity', 'smoking',
       'cardiac_rehab_eligibility', 'cholesterol_medication',
       'cardiac_rehab_particpation', 'population_age_65_plus', 'poverty_rate',
       'urban_rural_class_nonmetro'],
      dtype='object')

In [123]:
# penalized predictors

best_indices2 = [index for index,beta in enumerate(lasso_model.coef_) if beta == 0]
list(X.columns[best_indices2])

['diagnosed_diabetes',
 'obesity',
 'cholesterol_screen',
 'cardiac_rehab_sessions',
 'cardiac_rehab_units',
 'total_healthcare_cost_per_capita',
 'air_quality',
 'park_access',
 'population_total',
 'median_houshld_income',
 'urban_rural_class_large_fringe_metro',
 'urban_rural_class_small_metro']

In [115]:
# Evaluate model

r_squared = lasso_model.score(X_test_scaled, y_test)                       # R^2

y_predicted = lasso_model.predict(X_test_scaled)

test_error = mean_squared_error(y_test, y_predicted, squared=False)      # Root Mean Sqaured Error

print('R-squared' + ":", round(r_squared, 4))
print("predcition error:", round(test_error, 6))

R-squared: 0.9199
predcition error: 0.432442


**Model 3 (lassocv_model)**

The lassocv is just a lasso model with cross-valaidation. 

In [131]:
lassocv_model = LassoCV(alphas = alphas, cv = 5)
lassocv_model.fit(X_train_scaled, y_train);

In [132]:
# select best predictors
best_indices3 = [index for index,beta in enumerate(lassocv_model.coef_) if beta > 0 or beta < 0]
list(X.columns[best_indices3])

['cholesterol_awareness',
 'obesity',
 'leisure_inactivity',
 'smoking',
 'cardiac_rehab_eligibility',
 'cholesterol_medication',
 'cholesterol_screen',
 'cardiac_rehab_sessions',
 'cardiac_rehab_particpation',
 'cardiac_rehab_units',
 'total_healthcare_cost_per_capita',
 'air_quality',
 'population_age_65_plus',
 'population_total',
 'median_houshld_income',
 'poverty_rate',
 'urban_rural_class_large_fringe_metro',
 'urban_rural_class_nonmetro']

In [134]:
# penalized predictors
best_indices3 = [index for index,beta in enumerate(lassocv_model.coef_) if beta == 0]
list(X.columns[best_indices3])

['diagnosed_diabetes', 'park_access', 'urban_rural_class_small_metro']

In [133]:
# Evaluate model

r_squared = lassocv_model.score(X_test_scaled, y_test)                       # R^2

y_predicted = lassocv_model.predict(X_test_scaled)

test_error = mean_squared_error(y_test, y_predicted, squared=False)      # Root Mean Sqaured Error

print('R-squared' + ":", round(r_squared, 4))
print("predcition error:", round(test_error, 6))

R-squared: 0.9193
predcition error: 0.43397


**Discussion**

We can see that the Lasso has R-squared value of 0.9199 which is higher than that of the regular regression with value of 0.9173 as well of that of LassoCV with a value of 0.9193. Likewise, the prediction test error (0.432442) of the lasso model is better (i.e. lower) than than that of the regular linear regression model with a value of 0.439141 as well as  that of LassoCV with a value of 0.43397. Nevertheless, these differences are not very obvious. Since the Lasso models force some coefficients to zero ( 12 variables for lasso and only 3 for lassocv) thereby rendereing them insignificant, we will select the regular regression which accounts for all variables in this study since the differences in the evaluation metrics are not very much. Our decision for selecting this is also due to computational type which can quickly become too high as more data are introduced. Recall that in the regular model, we eliminated some four variables because of their high p-values. This assumption was validated by the fact that these four variables were also penalized in the Lasso model, and two of tem were penalized in the lassocv model.

Our selected model (mlr_model) can be represented as a linear combination of all the predictor variables to predict the value prevalance as follows:

This model predicts prevalence by an error of 0.4391 with 91.73% of variations in prevalence explained by the variations in the risk factors, healthcare opportunities and socio-economic factors.